In [36]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000) ## 800GB?

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics
import branca

import shared_utils

from siuba import *
import pandas as pd

import datetime as dt
import time
from zoneinfo import ZoneInfo

import importlib

import gcsfs
fs = gcsfs.GCSFileSystem()

from tqdm import tqdm_notebook
from tqdm.notebook import trange, tqdm

from IPython.display import display, Markdown


In [3]:
pd.set_option("display.max_columns", 100)

In [4]:
analysis_date = dt.date(2022,5,25)

In [5]:
itp_id = 300

In [6]:
#route = 3479

In [7]:
df = (tbl.views.gtfs_schedule_fact_daily_trips()
 >>filter(_.calitp_itp_id==itp_id)
 >>filter(_.service_date == analysis_date)
 >>filter(_.is_in_service == True)
      >> collect()
)

In [8]:
df

,feed_key,trip_key,trip_id,route_id,calitp_itp_id,calitp_url_number,service_id,service_date,service_indicator,service_start_date,service_end_date,service_inclusion,service_exclusion,is_in_service,calitp_extracted_at,calitp_deleted_at,n_stops,n_stop_times,trip_first_departure_ts,trip_last_arrival_ts,service_hours
0,4100255323842349957,-702563453614122743,883655,3501,300,0,40510,2022-05-25,0,2022-03-27,2022-08-13,True,None,True,2022-03-29,2099-01-01,14,14,58500,59700,0.333333
1,4100255323842349957,-4486545089448254498,882902,3491,300,0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2099-01-01,16,16,63900,65100,0.333333
2,4100255323842349957,7720324961420163680,883674,3501,300,0,40510,2022-05-25,0,2022-03-27,2022-08-13,True,None,True,2022-03-29,2099-01-01,17,17,33600,34620,0.283333
3,4100255323842349957,-2026554940129066198,882584,3489,300,0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2099-01-01,17,17,36420,37800,0.383333
4,4100255323842349957,-1277444804726595602,882638,3489,300,0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2099-01-01,17,17,78600,79800,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1511,4100255323842349957,-5305340657101354054,881503,3481,300,0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2099-01-01,39,39,62940,66600,1.016667
1512,4100255323842349957,-2768097702358623456,880798,3479,300,0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2099-01-01,45,45,28800,31800,0.833333
1513,4100255323842349957,3649527670215845467,880739,3479,300,0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2099-01-01,45,45,23100,25200,0.583333
1514,4100255323842349957,4943955253475173904,883303,3494,300,0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2099-01-01,48,48,30000,33120,0.866667


In [9]:
df.service_id.nunique()

6

In [10]:
df.service_id.value_counts()

10       1409
40510      52
77710      52
12010       1
92610       1
96910       1
Name: service_id, dtype: int64

In [11]:
(df
 >>filter(_.calitp_itp_id==itp_id)
 >>filter(_.service_date == analysis_date)
 >>filter(_.is_in_service == True)
 >> filter(_.calitp_extracted_at <= analysis_date, _.calitp_deleted_at >= analysis_date)
 >> summarize(unique= _.trip_id.nunique())
      
)

,unique
0,1516


In [12]:
(df
 >>filter(_.calitp_itp_id==itp_id)
 >>filter(_.service_date == analysis_date)
 >>filter(_.is_in_service == True)
 >> filter(_.calitp_extracted_at <= analysis_date, _.calitp_deleted_at >= analysis_date)
 >> group_by(_.route_id) 
 >> summarize(n=_.trip_id.nunique()) >> arrange(-_.n))

,route_id,n
0,3479,180
4,3485,153
8,3489,153
2,3481,145
9,3490,107
18,3501,104
12,3493,88
1,3480,82
15,3496,76
5,3486,71


In [13]:
rt = query_sql(f'''
SELECT * FROM `cal-itp-data-infra-staging.nla_views.test_rt_trips`
WHERE date = '{analysis_date}'
''')

/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/


In [14]:
rt

,calitp_itp_id,calitp_url_number,date,trip_id,vehicle_id
0,300,0,2022-05-25,880880,1300
1,300,0,2022-05-25,880776,1300
2,300,0,2022-05-25,880763,1300
3,300,0,2022-05-25,882786,1300
4,300,0,2022-05-25,880749,1300
...,...,...,...,...,...
1445,300,0,2022-05-25,881427,5319
1446,300,0,2022-05-25,883351,5320
1447,300,0,2022-05-25,881431,5320
1448,300,0,2022-05-25,881507,5320


In [15]:
df>>distinct(_.trip_id)

,trip_id
0,883655
1,882902
2,883674
3,882584
4,882638
...,...
1511,881503
1512,880798
1513,880739
1514,883303


In [16]:
df.sample()

,feed_key,trip_key,trip_id,route_id,calitp_itp_id,calitp_url_number,service_id,service_date,service_indicator,service_start_date,service_end_date,service_inclusion,service_exclusion,is_in_service,calitp_extracted_at,calitp_deleted_at,n_stops,n_stop_times,trip_first_departure_ts,trip_last_arrival_ts,service_hours
1138,4100255323842349957,6375932357001996899,883601,3500,300,0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2099-01-01,23,23,36420,37620,0.333333


In [17]:
len(df)

1516

In [18]:
join1 = df>>inner_join(_, rt, on = 'trip_id')

In [23]:
len(join1)

1450

In [79]:
join2 = (pd.merge(df, rt, how='outer', on='trip_id', indicator='have_rt'))

In [80]:
join2.have_rt.value_counts()

both          1450
left_only      168
right_only       0
Name: have_rt, dtype: int64

In [45]:
pct_ran = ('{:.1%}'.format((len(join2>>filter(_.have_rt=='left_only')))/(len(join2>>filter(_.have_rt=='both')))))

In [52]:
display(Markdown(f"On {analysis_date.strftime('%B %d')}, **{pct_ran}** "
                "of scheduled trips produced GTFS RT data"))

On May 25, **11.6%** of scheduled trips produced GTFS RT data

In [53]:
# checking the length of the string to see if there are any incomplete/null `trip_ids`

In [33]:
rt>>filter(_.trip_id.isnull())

,calitp_itp_id,calitp_url_number,date,trip_id,vehicle_id,str_len


In [28]:
rt['str_len'] = rt.trip_id.str.len()

In [34]:
rt>>arrange(-_.str_len)

,calitp_itp_id,calitp_url_number,date,trip_id,vehicle_id,str_len
0,300,0,2022-05-25,880880,1300,6
1,300,0,2022-05-25,880776,1300,6
2,300,0,2022-05-25,880763,1300,6
3,300,0,2022-05-25,882786,1300,6
4,300,0,2022-05-25,880749,1300,6
...,...,...,...,...,...,...
1445,300,0,2022-05-25,881427,5319,6
1446,300,0,2022-05-25,883351,5320,6
1447,300,0,2022-05-25,881431,5320,6
1448,300,0,2022-05-25,881507,5320,6


In [54]:
## Putting into a function

In [77]:
def find_pct_ran_trips():
    gtfs_daily = (
        tbl.views.gtfs_schedule_fact_daily_trips()
        >> filter(_.calitp_itp_id == itp_id)
        >> filter(_.service_date == analysis_date)
        >> filter(_.is_in_service == True)
        >> collect()
    )
    rt = query_sql(
        f"""
        SELECT * FROM `cal-itp-data-infra-staging.nla_views.test_rt_trips`
        WHERE date = '{analysis_date}'
        """
    )
    
    rt['str_len'] = rt.trip_id.str.len()
    
    #joinging 
    join = (pd.merge(gtfs_daily, rt, how='outer', on='trip_id', indicator='have_rt'))

    pct_ran = ('{:.1%}'.format((len(join2>>filter(_.have_rt=='left_only')))/(len(join2>>filter(_.have_rt=='both')))))
   
    return pct_ran

In [81]:
join2

,feed_key,trip_key,trip_id,route_id,calitp_itp_id_x,calitp_url_number_x,service_id,service_date,service_indicator,service_start_date,service_end_date,service_inclusion,service_exclusion,is_in_service,calitp_extracted_at,calitp_deleted_at,n_stops,n_stop_times,trip_first_departure_ts,trip_last_arrival_ts,service_hours,calitp_itp_id_y,calitp_url_number_y,date,vehicle_id,str_len,have_rt
0,4100255323842349957,-702563453614122743,883655,3501,300,0,40510,2022-05-25,0,2022-03-27,2022-08-13,True,None,True,2022-03-29,2099-01-01,14,14,58500,59700,0.333333,NaN,NaN,NaN,NaN,NaN,left_only
1,4100255323842349957,-4486545089448254498,882902,3491,300,0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2099-01-01,16,16,63900,65100,0.333333,300.0,0.0,2022-05-25,1346,6.0,both
2,4100255323842349957,7720324961420163680,883674,3501,300,0,40510,2022-05-25,0,2022-03-27,2022-08-13,True,None,True,2022-03-29,2099-01-01,17,17,33600,34620,0.283333,NaN,NaN,NaN,NaN,NaN,left_only
3,4100255323842349957,-2026554940129066198,882584,3489,300,0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2099-01-01,17,17,36420,37800,0.383333,300.0,0.0,2022-05-25,2116,6.0,both
4,4100255323842349957,-1277444804726595602,882638,3489,300,0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2099-01-01,17,17,78600,79800,0.333333,300.0,0.0,2022-05-25,2107,6.0,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1613,4100255323842349957,-5305340657101354054,881503,3481,300,0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2099-01-01,39,39,62940,66600,1.016667,300.0,0.0,2022-05-25,5319,6.0,both
1614,4100255323842349957,-2768097702358623456,880798,3479,300,0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2099-01-01,45,45,28800,31800,0.833333,300.0,0.0,2022-05-25,2118,6.0,both
1615,4100255323842349957,3649527670215845467,880739,3479,300,0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2099-01-01,45,45,23100,25200,0.583333,300.0,0.0,2022-05-25,2118,6.0,both
1616,4100255323842349957,4943955253475173904,883303,3494,300,0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2099-01-01,48,48,30000,33120,0.866667,300.0,0.0,2022-05-25,1802,6.0,both


In [35]:
## other queries

In [12]:
# (tbl.views.gtfs_rt_fact_daily_feeds
#  >>filter(_.calitp_itp_id==itp_id)
#  >>filter(_.date==analysis_date)
# )

In [23]:
# trips =(tbl.views.gtfs_schedule_dim_trips()
#  >>filter(_.calitp_itp_id==itp_id) >> collect())


In [24]:
#trips

In [15]:
#pd.merge(df, trips, on="route_id")

In [16]:
# (tbl.views.gtfs_schedule_fact_daily_trips()
#  >>filter(_.calitp_itp_id==itp_id)
#  >>filter(_.service_date == analysis_date)
# )

In [17]:
# (
#     tbl.views.gtfs_rt_fact_daily_feeds()
#     >> filter(_.calitp_itp_id == itp_id)
#     >> arrange(_.date ==analysis_date)
# )

In [18]:
# airtable_organizations = (
#     tbl.airtable.california_transit_organizations()
#     >> select(_.itp_id, _.name, _.caltrans_district,
#               _.website, _.ntp_id, _.drmt_organization_name)
#     >> filter(_.itp_id == itp_id)
#     >> collect()
# )

In [19]:
# airtable_organizations